# Importa Pacotes

In [1]:
import os
import re
import locale
import pandas as pd
from gestao import ConectarAccess

# Definição de variaveis e pré-produção

In [ ]:
# Variáveis Definidas
caminho_input = "C:/Users/vinir/Downloads/"
caminho_backup = "./Extrato da Conta Corrente/"

# Regex com o nome do arquivo, o primeiro grupo é o cartão e o segundo é mes da fatura
mask_arquivos = [
                 ('extrato.csv', '') 
                ]

# Informações de base de dados
tabela_extrato = '[extrato_cc]'
tabela_regra = '[regra_conversao]'
arquivo_base = 'GestaoFinanceira1'

#Variaveis de desenvolvimento
producao=True

print(f">> Execução em Produção: {producao}")
print(f">> Pasta de Leitura dos arquivos de entrada: {caminho_input}")
print(f">> Arquivo com a Base em Access: {arquivo_base}")
print(f">> Tabela Com o Extrato da Conta Corrente: {tabela_extrato}")
print(f">> Mascaras dos Arquivos: {', '.join([m[0] for m in mask_arquivos])}")

# Configuração do ambientes

In [ ]:
# Essa configuração é para o datetime entender os meses como mês Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

# Verificando arquivos presente na pasta

In [ ]:
files = []
for diretorio, subpastas, arquivos in os.walk(caminho_input):
    for arquivo in arquivos:
        for mask in mask_arquivos:
            if re.match(mask[0], arquivo):
                files.append(f"{diretorio}{arquivo}")
                print(f'[INFO] Localizado arquivo {arquivo}')

# Lendo Arquivos

In [6]:
colunas = [
            'Data_Lancamento',
            'Dependencia_Origem',
            'Historico',
            'Data_Balancete',
            'numero_documento',
            'Valor',
            'Col Vazia'
]

In [ ]:
df_files = []
for file in files:
    dados = pd.read_csv(file, names=colunas, encoding = "ISO-8859-1", skiprows=1)

    dados['Codigo'] = dados.index
    dados['Dependencia_Origem'] = dados['Dependencia_Origem'].fillna('')
    dados['Data_Balancete'] = dados['Data_Balancete'].fillna('')
    dados['Data_Lancamento'] = pd.to_datetime(dados['Data_Lancamento'], 
                                        format='%d/%m/%Y').apply(lambda x: x.strftime('%Y-%m-%d'))
    mes_extrato = dados.iloc[len(dados)-1].Data_Lancamento.replace('-','')[:6]
    dados['mes_extrato'] = mes_extrato
    print(f'[INFO] Mês do extrato: {mes_extrato}')

    df_files.append(dados)
    
    if producao:
        file_destino = f'{caminho_backup}{mes_extrato}.csv'
        print(f"[INFO] Movendo arquivo {file} para pasta {file_destino}")
        if os.path.exists(file_destino):
            os.remove(file_destino)
        os.rename(file, file_destino)
        

In [9]:
df = pd.concat(df_files)

df = df[[   'Codigo',
            'Data_Lancamento',
            'Dependencia_Origem',
            'Historico',
            'numero_documento',
            'Valor',
            'mes_extrato'
]]

# Ajustes dos dados
### Insere Tipo de Historico

In [10]:
# Lista de ajuste do Tipo de Historico
acc = ConectarAccess(arquivo_base)

ajuste_tipo = acc.select(f"""
        SELECT [chave], [valor]
        FROM {tabela_regra}
        WHERE [nome_regra] = 'Tipo_Lançamento_cc'
        """)
acc.close()

In [11]:
for ajs in ajuste_tipo:
    df.loc[df['Historico'].str.contains(ajs[0]), 'Tipo_Historico']=ajs[1]

df['Tipo_Historico'] = df['Tipo_Historico'].fillna('')

# Insere informação na Base de Dados

In [ ]:
# Instancia o objeto que faz as conexões com a base de dados
acc = ConectarAccess(arquivo_base)

# Verifica se existe dados na base e caso exista exclui
# Isso é para permitir reprocessamento

for mes in df['mes_extrato'].unique():
    print(f'[INFO] Inserindo informações para {mes}')
    verifica = acc.select(f"""
        SELECT COUNT(mes_extrato)
        FROM {tabela_extrato}
        WHERE mes_extrato = '{mes}' 
        """)[0][0]
    if verifica:
        print(f"[INFO] Dados presente para Arquivo {mes}... Deletando dados da base")
        acc.delete(f"DELETE FROM {tabela_extrato} WHERE mes_extrato = '{mes}'")

acc.insertAcc(df, tabela_extrato)
print('[INFO] Dados Inseridos com sucesso')

# Fecha a conexão
acc.close()

# Validação
### Validaçao do Saldo final

In [ ]:
for mes in df.mes_extrato.unique():
    d = df[df.mes_extrato == mes]

    if d[d.Historico == 'S A L D O'].Valor.iloc[0] - d.Valor.sum()/2 < 0.0001:
        print(f"Saldo verificado para Arquivo {mes}")